In [108]:
from addition_dataset import *
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_absolute_error, mean_squared_error
import torch.optim as optim
import numpy as np

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [2]:
# This class is used by the Trainer to get batches of data
class AdditionSubset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data
        
    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        # return x (input ids) and y (labels) as torch tensors
        return self.x_data[idx], self.y_data[idx]


class AdditionDataset(Dataset):   

    def __init__(self):
        self.subsets = {}

    def add_subset(self, name, x_data, y_data):
        self.subsets[name] = AdditionSubset(x_data, y_data)

    def get_subset(self, name):
        return self.subsets[name]
    
    def __str__(self):
        return f"subsets: {list(self.subsets.keys())}"
    
    def get_vocab_size(self):
        # 10 digits + '+', '=', '.', '#'
        return 14


In [3]:
char2ind_map = get_char2ind_map()
ind2char_map = make_ind2char_mapping(char2ind_map.copy())

x_train, y_train, x_val, y_val = get_addition_dataset(N=10000)

dataset = AdditionDataset()
dataset.add_subset('train', x_train, y_train)
dataset.add_subset('test', x_val, y_val)

# FIX: Wrap it in a DataLoader for batching
train_loader = DataLoader(dataset.get_subset('train'), batch_size=64, shuffle=True)
val_loader = DataLoader(dataset.get_subset('test'), batch_size=64, shuffle=True)

First 5/10000 expressions:
  ['4', '7', '+', '5', '1', '=', '9', '8', '.', '#']
  ['7', '5', '+', '9', '5', '=', '1', '7', '0', '.']
  ['3', '+', '1', '4', '=', '1', '7', '.', '#', '#']
  ['8', '2', '+', '9', '4', '=', '1', '7', '6', '.']
  ['2', '4', '+', '3', '1', '=', '5', '5', '.', '#']


In [4]:
dataset.get_subset('train')[0][0].shape

torch.Size([9])

In [5]:
teste = dataset.get_subset('train')
teste[1]

(tensor([ 3,  7, 10,  9,  6, 11,  1,  3,  3]),
 tensor([ 7, 10,  9,  6, 11,  1,  3,  3, 12]))

In [6]:
class SimpleRNN(nn.Module):

    def __init__(self, vocab_size, hidden_size):
        super(SimpleRNN, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        
        
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        
        
        self.lstm = nn.LSTM(hidden_size,hidden_size,batch_first=True)

        # Output layer
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        
        x = self.embedding(x)
        
        lstm_out, _ = self.lstm(x)
        
        output = self.fc(lstm_out)

        return output

In [7]:
def train_model(model, train_loader, val_loader, NUM_EPOCHS=10, learning_rate=0.001):
    
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss(ignore_index=13) # ingorar o #
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    final_loss_val = []
    final_loss_train = []

    for epoch in range(NUM_EPOCHS):
        model.train()
        running_train_loss = 0.0

        for batch_features, batch_labels in train_loader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

            optimizer.zero_grad()
            outputs = model(batch_features)
            
            outputs = outputs.permute(0, 2, 1)
            
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()

        train_loss = running_train_loss / len(train_loader)
        final_loss_train.append(train_loss)
        
        # --- Validation Loop ---
        model.eval() # Set the model to evaluation mode (e.g., disables dropout)
        running_val_loss = 0.0
        with torch.no_grad(): # Disable gradient calculation for validation
            for batch_features, batch_labels in val_loader:
                batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
                outputs = model(batch_features)
                outputs = outputs.permute(0, 2, 1)
                loss = criterion(outputs, batch_labels)
                running_val_loss += loss.item()

        val_loss = running_val_loss / len(val_loader)
        final_loss_val.append(val_loss)

        #if (epoch + 1) % 20 == 0 or epoch == 0: # Print more frequently for more epochs
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        
    return [final_loss_train, final_loss_val]


In [8]:
HIDDEN_SIZE = 128
NUM_EPOCHS = 150
LEARNING_RATE = 0.001
VOCAB_SIZE= dataset.get_vocab_size()

lstm_model = SimpleRNN(VOCAB_SIZE,HIDDEN_SIZE).to(device)

print(f"RNN Model parameters: {sum(p.numel() for p in lstm_model.parameters())}")

RNN Model parameters: 135694


In [9]:
lstm_losses = train_model(lstm_model,train_loader,val_loader,NUM_EPOCHS,LEARNING_RATE)

Epoch [1/150], Train Loss: 1.7010, Val Loss: 1.4114
Epoch [2/150], Train Loss: 1.3631, Val Loss: 1.3183
Epoch [3/150], Train Loss: 1.2923, Val Loss: 1.2604
Epoch [4/150], Train Loss: 1.2496, Val Loss: 1.2336
Epoch [5/150], Train Loss: 1.2163, Val Loss: 1.2017
Epoch [6/150], Train Loss: 1.1945, Val Loss: 1.1824
Epoch [7/150], Train Loss: 1.1706, Val Loss: 1.1653
Epoch [8/150], Train Loss: 1.1558, Val Loss: 1.1539
Epoch [9/150], Train Loss: 1.1394, Val Loss: 1.1314
Epoch [10/150], Train Loss: 1.1329, Val Loss: 1.1208
Epoch [11/150], Train Loss: 1.1131, Val Loss: 1.1152
Epoch [12/150], Train Loss: 1.1081, Val Loss: 1.1028
Epoch [13/150], Train Loss: 1.0940, Val Loss: 1.0937
Epoch [14/150], Train Loss: 1.0862, Val Loss: 1.0864
Epoch [15/150], Train Loss: 1.0787, Val Loss: 1.0988
Epoch [16/150], Train Loss: 1.0732, Val Loss: 1.0817
Epoch [17/150], Train Loss: 1.0669, Val Loss: 1.0771
Epoch [18/150], Train Loss: 1.0623, Val Loss: 1.0739
Epoch [19/150], Train Loss: 1.0578, Val Loss: 1.0606
Ep

In [88]:
def predict(modelo, equacao):
    
    char2ind_map = get_char2ind_map()
    ind2char_map = make_ind2char_mapping(char2ind_map.copy())
    equacao_tokens = [char2ind_map[c] for c in equacao]
    resultado = ""

    modelo.eval()

    with torch.no_grad():
        for _ in range(VOCAB_SIZE - len(equacao)):
            
            equacao_tokens_temp = equacao_tokens.copy()
            
            while len(equacao_tokens_temp) < VOCAB_SIZE:
                equacao_tokens_temp.append(13)
            
            output = modelo(torch.tensor([equacao_tokens_temp]).to(device))
            
            previsao = output[0, len(equacao_tokens) - 1, :]
            
            resposta_idx = torch.argmax(previsao).item()
            
            resposta = ind2char_map[resposta_idx]
            
            if resposta == '.':
                break
            
            resultado += resposta

            equacao_tokens.append(resposta_idx)

    return resultado

In [12]:
addition_expressions = make_addition_expressions(10000,2)
x_int_test, y_int_test = make_addition_samples_and_labels(addition_expressions.copy(), char2ind_map.copy())
x_str_test = convert_int2str(x_int_test.copy(), ind2char_map.copy())
y_str_test = convert_int2str(y_int_test.copy(), ind2char_map.copy())
problemas, respostas = split_sum_and_answer(x_str_test.copy())

print(problemas[0:5])

First 5/10000 expressions:
  ['1', '+', '6', '8', '=', '6', '9', '.', '#', '#']
  ['5', '9', '+', '5', '=', '6', '4', '.', '#', '#']
  ['9', '0', '+', '2', '2', '=', '1', '1', '2', '.']
  ['2', '5', '+', '1', '8', '=', '4', '3', '.', '#']
  ['3', '3', '+', '1', '7', '=', '5', '0', '.', '#']
[np.str_('1+68='), np.str_('59+5='), np.str_('90+22='), np.str_('25+18='), np.str_('33+17=')]


In [89]:
previsoes = []
labels = [x.replace('#','') for x in respostas]
labels = [x.replace('.','') for x in labels]
tamanho_erros = [0,0,0]
tamanhos = [0,0,0]
tokens_totais = 0
tokens_errados = 0
labels_temp = labels.copy()

for i in range(len(problemas)):

    # previspes dp modelo
    previsao = predict(lstm_model, str(problemas[i]))
    previsoes.append(previsao)
    
    # % de erros em cada calculo de diferentes digitos
    tamanhos[len(labels[i])-1]+=1
    if previsao != labels[i]:
        tamanho_erros[len(labels[i])-1]+=1
    
    # calculo do numero de tokens errados
    if previsao != labels_temp[i]:
        while(len(previsao) != len(labels_temp[i])):
            #print(i)
            if len(previsao) > len(labels_temp[i]):
                labels_temp[i]+='-'
            else:
                previsao+='-'
        #print(previsoes_temp[i])
        #print(labels_temp[i])
        for j in range(len(previsao)):
            # verificar cada token sem contar com os tokens '-' adicionados
            if (previsao[j] != labels_temp[i][j]) and previsao[j] != '-':
                tokens_errados +=1
    tokens_totais += len(previsao)
    
    print(f"Problem {i+1}: {problemas[i]}")
    print(f"Model Answer: {previsao}")
    print(f"Correct Answer: {labels[i]}")
    print("---")
    

Problem 1: 1+68=
Model Answer: 69
Correct Answer: 69
---
Problem 2: 59+5=
Model Answer: 65
Correct Answer: 64
---
Problem 3: 90+22=
Model Answer: 112
Correct Answer: 112
---
Problem 4: 25+18=
Model Answer: 43
Correct Answer: 43
---
Problem 5: 33+17=
Model Answer: 50
Correct Answer: 50
---
Problem 6: 34+81=
Model Answer: 115
Correct Answer: 115
---
Problem 7: 45+92=
Model Answer: 137
Correct Answer: 137
---
Problem 8: 44+27=
Model Answer: 71
Correct Answer: 71
---
Problem 9: 78+81=
Model Answer: 159
Correct Answer: 159
---
Problem 10: 86+88=
Model Answer: 174
Correct Answer: 174
---
Problem 11: 2+51=
Model Answer: 54
Correct Answer: 53
---
Problem 12: 26+24=
Model Answer: 50
Correct Answer: 50
---
Problem 13: 24+82=
Model Answer: 106
Correct Answer: 106
---
Problem 14: 79+21=
Model Answer: 100
Correct Answer: 100
---
Problem 15: 40+74=
Model Answer: 114
Correct Answer: 114
---
Problem 16: 14+62=
Model Answer: 76
Correct Answer: 76
---
Problem 17: 43+92=
Model Answer: 135
Correct Answer:

In [107]:
accuracy = accuracy_score(labels, previsoes)

print("\n--- Test Set Evaluation ---\n")
print(f"Accuracy: {accuracy:.4f}")
print(f"Token Accuracy: {tokens_errados/tokens_totais:.4f}% ({tokens_errados}/{tokens_totais})")
print(f"Cross Entropy Loss: \n\t Train:\n\t\t Initial loss: {lstm_losses[0][0]:.4f} \n\t\t Final loss:   {lstm_losses[0][-1]:.4f} \
        \n\t Test:\n\t\t Initial loss: {lstm_losses[1][0]:.4f} \n\t\t Final loss:   {lstm_losses[1][-1]:.4f}")
print(f"% of Errors in Calculations:\n\t single digit answers: {100*tamanho_erros[0]/tamanhos[0]:.4f}% ({tamanho_erros[0]}/{tamanhos[0]}) \
      \n\t double digit answers: {100*tamanho_erros[1]/tamanhos[1]:.4f}%  ({tamanho_erros[1]}/{tamanhos[1]}) \
      \n\t triple digit answers: {100*tamanho_erros[2]/tamanhos[2]:.4f}%  ({tamanho_erros[2]}/{tamanhos[2]})")



--- Test Set Evaluation ---

Accuracy: 0.9679
Token Accuracy: 0.0170% (423/24876)
Cross Entropy Loss: 
	 Train:
		 Initial loss: 1.7010 
		 Final loss:   0.8261         
	 Test:
		 Initial loss: 1.4114 
		 Final loss:   0.8804
% of Errors in Calculations:
	 single digit answers: 61.0169% (36/59)       
	 double digit answers: 4.8548%  (244/5026)       
	 triple digit answers: 0.8342%  (41/4915)
